# Read property data

In [31]:
# 这里爬的是所有房子的数据，下面路径改一下
import pandas as pd
import re
import math as math
from itertools import permutations 
property_data_2022 = pd.read_json('../data/raw/2022_house_data.json')

In [32]:
property_data_2022.head()

,https://www.domain.com.au/101-25-33-wills-street-melbourne-vic-3000-14868152,https://www.domain.com.au/2905b-11-rose-lane-melbourne-vic-3000-16074279,https://www.domain.com.au/1901-368-st-kilda-road-melbourne-vic-3000-16073890,https://www.domain.com.au/1211-200-spencer-street-melbourne-vic-3000-16073870,https://www.domain.com.au/1008-380-little-lonsdale-street-melbourne-vic-3000-16072582,https://www.domain.com.au/3-27-flinders-lane-melbourne-vic-3000-16072122,https://www.domain.com.au/611-408-lonsdale-street-melbourne-vic-3000-16071839,https://www.domain.com.au/102-9-bligh-place-melbourne-vic-3000-16071806,https://www.domain.com.au/1715-220-spencer-street-melbourne-vic-3000-16071670,https://www.domain.com.au/104-300-swanston-street-melbourne-vic-3000-13671355,...,https://www.domain.com.au/6-2-4-pymble-avenue-inverloch-vic-3996-15660207,https://www.domain.com.au/6-bruce-street-inverloch-vic-3996-16071116,https://www.domain.com.au/16-kenneth-street-inverloch-vic-3996-15315976,https://www.domain.com.au/25a-veronica-street-inverloch-vic-3996-16006402,https://www.domain.com.au/16b-sandy-mount-avenue-inverloch-vic-3996-16006035,https://www.domain.com.au/28-beachcomber-drive-inverloch-vic-3996-15920343,https://www.domain.com.au/14-inverloch-parade-inverloch-vic-3996-16040296,https://www.domain.com.au/10-hopetoun-street-inverloch-vic-3996-15935242,https://www.domain.com.au/inverloch-vic-3996-14434367,https://www.domain.com.au/64-dixon-street-inverloch-vic-3996-15951739
name,101/25-33 Wills Street Melbourne VIC 3000,2905B/11 Rose Lane Melbourne VIC 3000,1901/368 St Kilda Road Melbourne VIC 3000,1211/200 Spencer Street Melbourne VIC 3000,1008/380 Little Lonsdale Street Melbourne VIC ...,3/27 Flinders Lane Melbourne VIC 3000,611/408 Lonsdale Street Melbourne VIC 3000,102/9 Bligh Place Melbourne VIC 3000,1715/220 Spencer Street Melbourne VIC 3000,104/300 Swanston Street Melbourne VIC 3000,...,6/2-4 Pymble Avenue Inverloch VIC 3996,6 Bruce Street Inverloch VIC 3996,16 Kenneth street Inverloch VIC 3996,25A Veronica Street Inverloch VIC 3996,16B Sandy Mount Avenue Inverloch VIC 3996,28 Beachcomber Drive Inverloch VIC 3996,14 Inverloch Parade Inverloch VIC 3996,10 Hopetoun Street Inverloch VIC 3996,Inverloch VIC 3996,64 Dixon Street Inverloch VIC 3996
cost_text,$400,$575 per week,$1800 Per Week,$480 per week,$400 pw,$420 per week,$350,$600,$600,$550,...,$420 wk,$400,$470 wk,400 per week,$400.00 per week,$550 wk,$420 wk,$370 wk,$475 wk,Contact Agent
property_type,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,Apartment / Unit / Flat,...,Apartment / Unit / Flat,House,House,House,House,House,House,House,House,House
property_feature,"[Friday, 7th October 2022, $1738]","[Tuesday, 20th September 2022, $2499]","[Available Now, $10800]","[Tuesday, 18th October 2022, $2085]","[Friday, 9th September 2022, $1738]","[Wednesday, 28th September 2022, $1825]",[Available Now],"[Available Now, $2608]","[Monday, 12th September 2022, $2607]","[Available Now, $2390]",...,"[Available Now, $1825]","[Monday, 19th September 2022, $1733, 625.7m²]","[Wednesday, 14th September 2022, $2050]","[Available Now, $1738]","[Available Now, $1738]","[Available Now, $2390]","[Available Now, $1820]","[Available Now, $1610]","[Available Now, $2065]","[Available Now, 200m²]"
coordinates,"[-37.8107757, 144.9569269]","[-37.815446, 144.9541165]","[-37.8318086, 144.9710272]","[-37.816228, 144.9532465]","[-37.8121026, 144.9581291]","[-37.8147317, 144.9733655]","[-37.8125979, 144.9604012]","[-37.817317, 144.96371]","[-37.8157157, 144.9529559]","[-37.8106088, 144.9645798]",...,"[-38.6332252, 145.7367946]","[-38.6303499, 145.7311203]","[-38.629687, 145.7322922]","[-38.6428993, 145.7101579]","[-38.6340011, 145.725239]","[-38.6412502, 145.7031282]","[-38.6287143, 145.7358082]","[-38.6341243, 145.7306862]","[-38.6314613, 145.

In [33]:
property_data_2022 = property_data_2022.T
property_data_2022 = property_data_2022.reset_index(drop=True)

In [34]:
property_data_2022.head()

,name,cost_text,property_type,property_feature,coordinates,rooms,desc
0,101/25-33 Wills Street Melbourne VIC 3000,$400,Apartment / Unit / Flat,"[Friday, 7th October 2022, $1738]","[-37.8107757, 144.9569269]","[[1 Bed], [1 Bath], []]",Mary Khudhur
1,2905B/11 Rose Lane Melbourne VIC 3000,$575 per week,Apartment / Unit / Flat,"[Tuesday, 20th September 2022, $2499]","[-37.815446, 144.9541165]","[[2 Beds], [1 Bath], [1 Parking]]",Mikaele Horsnell
2,1901/368 St Kilda Road Melbourne VIC 3000,$1800 Per Week,Apartment / Unit / Flat,"[Available Now, $10800]","[-37.8318086, 144.9710272]","[[3 Beds], [2 Baths], [2 Parking]]",Darren McMullin
3,1211/200 Spencer Street Melbourne VIC 3000,$480 per week,Apartment / Unit / Flat,"[Tuesday, 18th October 2022, $2085]","[-37.816228, 144.9532465]","[[1 Bed], [1 Bath], [1 Parking]]",South Melbourne Leasing
4,1008/380 Little Lonsdale Street Melbourne VIC ...,$400 pw,Apartment / Unit / Flat,"[Friday, 9th September 2022, $1738]","[-37.8121026, 144.9581291]","[[1 Bed], [1 Bath], []]",Shaun Broadbent


# Get postcode from address

In [35]:
def extract_postcode (row):
    split_word = 'VIC'
    postcode = row['name'].partition(split_word)[2]
    postcode = postcode.strip()
    return postcode

In [36]:
# Checking the postcode
property_data_2022.apply (lambda row: extract_postcode(row), axis=1)

0        3000
1        3000
2        3000
3        3000
4        3000
         ... 
15087    3996
15088    3996
15089    3996
15090    3996
15091    3996
Length: 15092, dtype: object

In [37]:
# Assigning the postcode
property_data_2022["postcode"] = property_data_2022.apply (lambda row: extract_postcode(row), axis=1)

# Scrapping the SA2 name matches to postcodes
# 爬和邮编对应的 SA2 name

In [38]:
# getting the the table postcodes matches to SA2 region name
# The data scrape from https://www.matthewproctor.com/full_australian_postcodes_vic
import requests
from bs4 import BeautifulSoup
url = "https://www.matthewproctor.com/full_australian_postcodes_vic" #网址
data = requests.get(url).text  #获取数据
soup = BeautifulSoup(data, 'html.parser') #获取数据中想要的东西

In [39]:
table = soup.find("table", {"class": "table table-striped"}) #获取其中的表格

In [40]:
df = pd.DataFrame(columns=['postcodes', 'SA2_names'])

# Collecting Ddata
for row in table.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        postcode = columns[1].text.strip()
        SA2_name = columns[19].text.strip()
        df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1753/3783446249.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1753/3783446249.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1753/3783446249.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1753/3783446249.py:11: FutureWarning: The frame

In [41]:
sa2_dataframe = df.iloc[1:]

In [42]:
# checking the results we scrapped
sa2_dataframe.head()

,postcodes,SA2_names
1,3000,Melbourne
2,3001,Melbourne
3,3002,East Melbourne
4,3003,West Melbourne
5,3004,Southbank


# Adding the SA2 name to each postcode

In [43]:
def add_sa2(row,sa2_dataframe):
    postcode = row['postcode']
    selected_postcode = sa2_dataframe.loc[sa2_dataframe['postcodes'] == postcode]
    sa2 = [(selected_postcode.iloc[i])["SA2_names"] for i in range(len(selected_postcode))]
    sa2=list(set(sa2))
    return sa2

In [44]:
property_data_2022.apply(lambda row: add_sa2(row,sa2_dataframe), axis=1)

0                            [Melbourne]
1                            [Melbourne]
2                            [Melbourne]
3                            [Melbourne]
4                            [Melbourne]
                      ...               
15087    [Wonthaggi - Inverloch, Foster]
15088    [Wonthaggi - Inverloch, Foster]
15089    [Wonthaggi - Inverloch, Foster]
15090    [Wonthaggi - Inverloch, Foster]
15091    [Wonthaggi - Inverloch, Foster]
Length: 15092, dtype: object

In [45]:
property_data_2022["sa2_region"] = property_data_2022.apply(lambda row: add_sa2(row,sa2_dataframe), axis=1)

# Starting the data preprocssing:(2022 data)
1. extract the cost out,remove all the data which does not have a valid rent cost,找到所有per week或者pw前面的数字
2. seperate the number of bedroom, number bathroom and number of parking into an individual new column （✅）
3. Adding the SA2 population 2022
4. Adding the SA2 Income 2022
5. Adding the distance to the hospital.
6. Adding the distance to school.
7. 创建新的一列 apartment/unit/flat和house然后用0,1表示. (如果需要的话)


In [46]:
def extract_cost_number (row):
    cost_string = row['cost_text']
    
    # 替换 / 为 per
    cost_string = cost_string.replace('/', ' per ')
    
    cost_list = []
    
    # 找出可能的租金字符串
    for cost in re.findall('\d+\s*\w+\s*\w+',cost_string):
        cost_list.append(cost)
        
    cost_list = [cost.lower() for cost in cost_list]
    
    # 替换简写
    dictOfStrings = {'pw': 'per week', 'weekly': 'week',
                    'pcm':'per month','mth':'month','wek':'week',
                    'pa':'per year','wk':'week','annually':'year',
                    'pwk':'per week','poa':'per year','monthly':'month',
                    'pcw':'per week','weekly':'week','annum':'year','pm':'per month'}
    replace_abb = []
    for cost in cost_list:
        for word, replacement in dictOfStrings.items():
            cost = re.sub(word, replacement, cost)
        replace_abb.append(cost)     
    if len(replace_abb) > 0:
        cost_type = replace_abb[0]
        # 替换单字母简写
        dictOfAlpha = {'h':'hour','d':'day','w':'week','m':'month','y':'year'}
        processed_test = ''
        for w in cost_type.split():
            if (len(w) == 1) and (w in dictOfAlpha):
                processed_test += dictOfAlpha[w]
            else:
                processed_test += w
            processed_test += " "
        rent_type = ''.join([i for i in processed_test if not i.isdigit()])
        cost = ''.join([i for i in processed_test if i.isdigit()])
        if 'hour' in rent_type:
            rent_per_week = float(cost)*24*7
        elif 'day' in rent_type:
            rent_per_week = float(cost)*7    
        elif 'month' in rent_type:
            rent_per_week = (float(cost)/30)*7   
        elif 'year' in rent_type:
            rent_per_week = (float(cost)/365)*7  
        elif 'fortnightly' in rent_type:
            rent_per_week = (float(cost)/2)  
        else:
            rent_per_week = float(cost)
        return rent_per_week
    else:
        return "No Available"  

In [55]:
#add the weekly rent into a new column
property_data_2022['week_rent'] = property_data_2022.apply (lambda row: extract_cost_number(row), axis=1)

In [58]:
def data_preprocessing_22(data_set):
    #extract the price per week out
    #现在问题是有很多数据的价格都有无用的信息
    bed_num=[]
    bath_num=[]
    park_num=[]
    #seperate the bedroom, bathroom and parking area
    for value in data_set['rooms']:
        bed_num.append(re.findall(r'(\d+) Bed', str(value))) #找到bed前面的数字
        bath_num.append(re.findall(r'(\d+) Bath', str(value)))
        park_num.append(re.findall(r'(\d+) Parking', str(value)))
    data_set['bedroom_num'] = bed_num
    data_set['bathroom_num'] = bath_num
    data_set['parking_num'] = park_num
    data_set = data_set.drop(['cost_text'],axis=1)
    return data_set
property_data_2022=data_preprocessing_22(property_data_2022) 


In [60]:
# use the first SA2 area as the area which the house belows to 
#如果找到了每一个区域的话，可以把每一个区域的的code和经纬度坐标搞成一个字典，然后房子的经纬度坐标分别和每个区域的经纬度坐标算一个距离，并且把房子分配给最近的SA2区域中心
SA2_centroids = pd.read_csv('../data/raw/mb_2016_centroids.csv')
all_value = []
SA2_vic = SA2_centroids[SA2_centroids['STE_NAME16']=='Victoria']
SA2_vic = SA2_vic[['SA2_MAIN16','SA2_NAME16','xcoord','ycoord']]
SA2_vic.head()
SA2_vic_xcoord = SA2_vic.groupby('SA2_NAME16')["xcoord"].mean()
SA2_vic_ycoord = SA2_vic.groupby('SA2_NAME16')["ycoord"].mean()
#现在拿到了所有SA2的中心区域
SA2_vic_xy = pd.merge(SA2_vic_xcoord,SA2_vic_ycoord,left_on = 'SA2_NAME16',right_on = 'SA2_NAME16')
#groupy the SA2_NAME16 and take the average of xcoord and ycoord
SA2_vic_xy=dict(SA2_vic_xy.transpose())
all_co = []
for value in property_data_2022['sa2_region']:
    ind_co=[] #每一行的结果
    for region in value:
        #print(region)
        if region in SA2_vic_xy.keys():
            ind_co.append(SA2_vic_xy[region])
    all_co.append(ind_co)
property_data_2022['sa2_coor'] = all_co
property_data_2022.head()

,name,property_type,property_feature,coordinates,rooms,desc,postcode,sa2_region,bedroom_num,bathroom_num,parking_num,sa2_coor,week_rent
0,101/25-33 Wills Street Melbourne VIC 3000,Apartment / Unit / Flat,"[Friday, 7th October 2022, $1738]","[-37.8107757, 144.9569269]","[[1 Bed], [1 Bath], []]",Mary Khudhur,3000,[Melbourne],[1],[1],[],"[[144.9627648781132, -37.813376920301884]]",400.0
1,2905B/11 Rose Lane Melbourne VIC 3000,Apartment / Unit / Flat,"[Tuesday, 20th September 2022, $2499]","[-37.815446, 144.9541165]","[[2 Beds], [1 Bath], [1 Parking]]",Mikaele Horsnell,3000,[Melbourne],[2],[1],[1],"[[144.9627648781132, -37.813376920301884]]",575.0
2,1901/368 St Kilda Road Melbourne VIC 3000,Apartment / Unit / Flat,"[Available Now, $10800]","[-37.8318086, 144.9710272]","[[3 Beds], [2 Baths], [2 Parking]]",Darren McMullin,3000,[Melbourne],[3],[2],[2],"[[144.9627648781132, -37.813376920301884]]",1800.0
3,1211/200 Spencer Street Melbourne VIC 3000,Apartment / Unit / Flat,"[Tuesday, 18th October 2022, $2085]","[-37.816228, 144.9532465]","[[1 Bed], [1 Bath], [1 Parking]]",South Melbourne Leasing,3000,[Melbourne],[1],[1],[1],"[[144.9627648781132, -37.813376920301884]]",480.0
4,1008/380 Little Lonsdale Street Melbourne VIC ...,Apartment / Unit / Flat,"[Friday, 9th September 2022, $1738]","[-37.8121026, 144.9581291]","[[1 Bed], [1 Bath], []]",Shaun Broadbent,3000,[Melbourne],[1],[1],[],"[[144.9627648781132, -37.813376920301884]]",400.0


In [61]:
#function to calculate the distance 
def getDistance(latA, lonA, latB, lonB):
    ra = 6378140  # 赤道半徑
    rb = 6356755  # 極半徑
    flatten = (ra - rb) / ra  # Partial rate of the earth
    # change angle to radians
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)

    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB)) ** 2 / math.cos(x / 2) ** 2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB)) ** 2 / math.sin(x / 2) ** 2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)
    distance = round(distance / 1000, 4)
    return distance

In [ ]:
#根据房子和SA2区域中心的距离决定每个房子属于哪一个SA2区域
#返回出最近的SA2区域
#coordinates是房子的位置，每一趟房子固定有一组数据
#sa2_coor是好几个sa2区域中心的位置，需要将一套房子的位置和每一个区域中心的位置返回出一个距离
def allocate_sa2_region(data_set):
    #这个是对于每一个房子到所有sa2的距离
    all_distance = []
    for value1 in data_set['coordinates']:
        #单独一个房子到sa2的距离
        one_distance = []
        for value2 in data_set['sa2_coor']:
            one_distance.append(getDistance(value1[0],value1[1],value[1],value[0]))
    allocated_region = []
    #all_distance 应该形式是 [[房1距离1，房1距离2],[房2距离1,房2距离2]]
    #循环all_distance里面所有的小list，找到最小的那个距离的index，返回对应行的sa2区域的那个index并且加到一个房子的region里,然后把每个一个房子最近的region加入到所有区域房子最近的region里面
    
    all_region = []
    




In [107]:
a=[[144.9627648781132, -37.813376920301884],[144,-37]] #房子所属的两个sa2区域的中心坐标
b= [1,2] #房子的坐标
two_distance = []
for value in a:
    two_distance.append(getDistance(b[0],b[1],value[0],value[1]))
    #找到对于每个房子来说，到每个区域的距离，这个循环有点刁钻


[5748.5512, 5751.5129]

In [122]:
#Find all the distance with the SA2 centriod to each house using the index and append that SA2 in that index as well
all_distance = []
for value1 in property_data_2022['coordinates']:
    for value2 in property_data_2022['sa2_coor']:
        all_distance.append(getDistance(value1[0],value1[1],value[1],value[0]))


TypeError: cannot convert the series to <class 'float'>

In [53]:
property_data_2022.to_csv('../data/raw/2022_house_data.csv')